In [1]:
import numpy as np
import pandas as pd
import scipy.stats
import random
from scipy.spatial.distance import cdist
from gurobipy import *
import matplotlib.pyplot as plt

In [2]:
myModel = Model('project')

Academic license - for non-commercial use only


In [3]:
#Generating affinity matrix C, using department partition as shown by Huseyin

random.seed(1)
profs = 30 #num professors
num_departments = 5 #num departments
preference_prob = .5 #prob of professor i within department d liking professor j within same department
 
C = np.zeros((profs, profs))

random.seed(1)
#partitions = np.random.randint(0,profs,size = num_departments - 1)
partitions = [5, 9, 16, 23]
partitions.sort()
partitions = np.append(partitions,profs)


# last_p = 0
# for p in partitions:
#     C[last_p:p,last_p:p] = np.random.binomial(1,preference_prob, size = (p - last_p,p - last_p))
#     last_p = p

last_p = 0
for p in partitions:
    for prof in range(last_p, p):
        affinities = np.random.randint(last_p,p,size = 2)
        C[prof, affinities] = 1
    last_p = p

for i in range(0,profs):
    C[i,i] = 1
    
# print(partitions)
# print(C)

In [4]:
#generating room preferences

#hard-coded room numbers
second_floor_rooms = [210,212,214,216,220,252,254,255,
                      256,257,259,261,262,263, 264,265,
                      266,268,270,271,273, 291, 293]
    
third_floor_rooms = [int('3' + str(num)[1:]) for num in second_floor_rooms]
third_floor_rooms.remove(391) # remove 391 393
third_floor_rooms.remove(393)
room_list = second_floor_rooms + third_floor_rooms
rooms = len(room_list)
shuffled_rooms = random.sample(room_list, rooms)
num_rankings = 3

#R_ is num_professors x num_ranking matrix, R_ik is the kth choice room number for professor i
R_ = np.zeros((profs,num_rankings))

#uniform case: generates room rankings for each professor, assuming all rooms are equally preferred 
# for i in range(profs):
#     R_[i] = np.random.choice(room_list, size=num_rankings, replace=False, p=None)

#weighted case: generates room rankings for each professor, assuming room preferences follow a geometric distribution ~geom(p)  
p = .2
room_probs = [scipy.stats.geom.pmf(k,p) for k in range(1,rooms+1)]
room_probs[-1] = 1 - min(sum(room_probs[:-1]),1)

for i in range(profs):
    R_[i] = np.random.choice(shuffled_rooms, size=num_rankings, replace=False, p=room_probs)
    #print(R_[i])
    
#R1 is num_professors x num_room X num_ranking matr ix, 
#R_ijk is 1 if professor i's kth ranked room is the jth element of room_list
R1= np.zeros((profs,rooms,num_rankings))
for i in range(profs):
    for j in range(rooms):
        for k in range(num_rankings):
            if R_[i,k]==shuffled_rooms[j]:
                R1[i,j,k]=1

# R in Frazer's part, R_30*44, R_ij means if professor i likes room j, and it's leveled 3 to 1
R = 3*R1[:,:,0] + 2*R1[:,:,1]+ R1[:,:,2]

In [10]:
# import the distance matrix

Distance = pd.read_csv("huddle_distance_csv.csv", index_col= 0, sep=";")
Distance.fillna(value=0, inplace=True)

D = np.array(Distance)

In [11]:
# start building optimization model
myModel = Model('huddle_project')

In [12]:
# add variablies x_ij and y_njmk

myVars = {}
for i in range(profs):
    for j in range(rooms):
        curVar = myModel.addVar( vtype = GRB.BINARY , ub=1 , name = 'x' + str(i) + '_' + str(room_list[j]) )
        myVars['x' + str(i) + '_' + str(room_list[j])] = curVar
for n in range(profs):
    for m in range(profs):
        for j in range(rooms):
            for k in range(rooms):
                curVar = myModel.addVar( vtype = GRB.BINARY , ub = 1 , name = 'y' + str(n) + '_' + str(room_list[j])+ '_' + str(m) + '_' + str(room_list[k]))
                myVars['y' + str(n) + '_' + str(room_list[j])+ '_' + str(m) + '_' + str(room_list[k])] = curVar
myModel.update()

In [13]:
# obj

objExpr = LinExpr()

for i in range(profs):
    for j in range(rooms):
        curVar = myVars['x' + str(i) + '_' + str(room_list[j])]
        objExpr += R[i][j] * curVar

for n in range(profs):
    for m in range(profs):
        for j in range(rooms):
            for k in range(rooms):
                curVar = myVars['y' + str(n) + '_' + str(room_list[j])+ '_' + str(m) + '_' + str(room_list[k])]
                objExpr += (-1) * D[j][k] * C[m][n] * curVar
                
myModel.setObjective(objExpr, GRB.MAXIMIZE)

In [15]:
# constraints

# 1 room should be assigned to no more than 1 professor
for j in range(rooms):
    constrExpr = LinExpr()
    for i in range(profs):
        curVar = myVars['x' + str(i) + '_' + str(room_list[j])]
        constrExpr += curVar
    myModel.addConstr(lhs = constrExpr, sense = GRB.LESS_EQUAL , rhs = 1)

# 1 professor must be assigned to a room
for i in range(profs):
    constrExpr = LinExpr()
    for j in range(rooms):
        curVar = myVars['x' + str(i) + '_' + str(room_list[j])]
        constrExpr += curVar
    myModel.addConstr(lhs = constrExpr, sense = GRB.EQUAL , rhs = 1)

# constraints between y_njmk and x_ij
for n in range(profs):
    for j in range(rooms):
        for m in range(profs):
            for k in range(rooms):
                # y_njmk <= x_nj
                constrExpr1 = LinExpr()
                constrExpr1 += myVars['y' + str(n) + '_' + str(room_list[j])+ '_' + str(m) + '_' + str(room_list[k])]
                constrExpr1 -= myVars['x' + str(n) + '_' + str(room_list[j])]
                myModel.addConstr(lhs = constrExpr1, sense = GRB.LESS_EQUAL , rhs = 0) 
                # y_njmk <= x_mk
                constrExpr2 = LinExpr()
                constrExpr2 += myVars['y' + str(n) + '_' + str(room_list[j])+ '_' + str(m) + '_' + str(room_list[k])]
                constrExpr2 -= myVars['x' + str(m) + '_' + str(room_list[k])]
                myModel.addConstr(lhs = constrExpr2, sense = GRB.LESS_EQUAL , rhs = 0) 
                # x_nj + x_mk - y_njmk <= 1
                constrExpr3 = LinExpr()
                constrExpr3 += myVars['x' + str(n) + '_' + str(room_list[j])]
                constrExpr3 += myVars['x' + str(m) + '_' + str(room_list[k])]
                constrExpr3 -= myVars['y' + str(n) + '_' + str(room_list[j])+ '_' + str(m) + '_' + str(room_list[k])]
                myModel.addConstr(lhs = constrExpr3, sense = GRB.LESS_EQUAL , rhs = 1)
myModel.update()     
                

In [16]:
myModel.write(filename = 'project.lp')

In [ ]:
myModel.optimize()

Optimize a model with 5227318 rows, 1743720 columns and 12199440 nonzeros
Variable types: 0 continuous, 1743720 integer (1743720 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2684 rows and 0 columns (presolve time = 7s) ...
Presolve removed 4004 rows and 1320 columns (presolve time = 10s) ...
Presolve removed 4004 rows and 1320 columns (presolve time = 15s) ...
Presolve removed 12766 rows and 1320 columns (presolve time = 20s) ...
Presolve removed 12766 rows and 1320 columns (presolve time = 25s) ...
Presolve removed 12766 rows and 1320 columns (presolve time = 30s) ...
Presolve removed 12766 rows and 1320 columns (presolve time = 35s) ...
Presolve removed 12766 rows and 1320 columns (presolve time = 40s) ...
Presolve removed 12766 rows and 1320 columns (presolve time = 45s) ...
Presolve removed 12766 rows and 1320 columns (presolve time = 50s) ...
